In [31]:
import numpy as np
import pandas as pd
from PDB import PDB
import os
import copy
from json import loads

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def read_scores(file, key='decoy'):
    """Take a json formatted score.sc file and incorporate into dictionary of dictionaries with 'key' value as outer key

    Args:
        file (str): Location on disk of scorefile
    Keyword Args:
        key='decoy' (str): Name of the json key to use as outer dictionary identifier
    Returns:
        score_dict (dict): {design_name: {all_score_metric_keys: all_score_metric_values}, ...}
    """
    with open(file, 'r') as f:
        score_dict = {}
        for score in f.readlines():
            entry = loads(score)
            design = entry[key]  # entry['decoy'].split('_')[-1]
            if design not in score_dict:
                score_dict[design] = entry
            else:
                # to ensure old trajectories don't have lingering protocol info TODO clean Rosetta protocol generation
                for protocol in protocols:
                    if protocol in entry.keys():
                        for rm_protocol in protocols:
                            try:
                                score_dict[design].pop(rm_protocol)
                            except KeyError:
                                pass
                score_dict[design].update(entry)

    return score_dict

In [4]:
residue_template = {'energy': {'complex': 0, 'unbound': 0, 'fsp': 0, 'cst': 0},
                    'sasa': {'polar': {'complex': 0, 'unbound': 0}, 'hydrophobic': {'complex': 0, 'unbound': 0},
                             'total': {'complex': 0, 'unbound': 0}},
                    'type': None, 'hbond': 0, 'core': 0, 'interior': 0, 'rim': 0, 'support': 0}  # , 'hot_spot': 0}

In [5]:
def dirty_dirty_residue_processing(score_dict, offset=None, hbonds=None):  # mutations,
    """Process Residue Metrics from Rosetta score dictionary

    One-indexed residues
    Args:
        score_dict (dict): {'0001': {'buns': 2.0, 'per_res_energy_15': -3.26, ...,
                            'yhh_planarity':0.885, 'hbonds_res_selection': '15A,21A,26A,35A,...'}, ...}
        mutations (dict): {'reference': {mutation_index: {'from': 'A', 'to: 'K'}, ...},
                           '0001': {mutation_index: {}, ...}, ...}
    Keyword Args:
        offset=None (dict): {'A': 0, 'B': 102}
        hbonds=None (dict): {'0001': [34, 54, 67, 68, 106, 178], ...}
    Returns:
        residue_dict (dict): {'0001': {15: {'type': 'T', 'energy_delta': -2.771, 'bsa_polar': 13.987, 'bsa_hydrophobic': 
            22.29, 'bsa_total': 36.278, 'hbond': 0, 'core': 0, 'rim': 1, 'support': 0}, ...}, ...}  # , 'hot_spot': 1
    """
    total_residue_dict = {}
    for entry in score_dict:
        residue_dict = {}
        # for column in columns:
        for key, value in score_dict[entry].items():
            # metadata = column.split('_')
            if key.startswith('per_res_'):
                metadata = key.split('_')
                res = int(metadata[-1])
                r_type = metadata[2]  # energy or sasa
                pose_state = metadata[-2]  # oligomer or complex
                if pose_state == 'unbound' and offset:
                    res += offset[metadata[-3]]  # get oligomer chain offset
                if res not in residue_dict:
                    residue_dict[res] = copy.deepcopy(residue_template)
                if r_type == 'sasa':
                    # Ex. per_res_sasa_hydrophobic_1_unbound_15 or per_res_sasa_hydrophobic_complex_15
                    polarity = metadata[3]
                    residue_dict[res][r_type][polarity][pose_state] = round(score_dict[entry][key], 3)
                    # residue_dict[res][r_type][polarity][pose_state] = round(score_dict[entry][column], 3)
                else:
                    # Ex. per_res_energy_1_unbound_15 or per_res_energy_complex_15
                    residue_dict[res][r_type][pose_state] = round(score_dict[entry][key], 3)
        # if residue_dict:
#         for res in residue_dict:
# #             try:
# #                 residue_dict[res]['type'] = mutations[entry][res]
# #             except KeyError:
# #                 residue_dict[res]['type'] = mutations['reference'][res]  # fill with aa from wild_type sequence
#             if hbonds:
#                 if res in hbonds[entry]:
#                     residue_dict[res]['hbond'] = 1
#             residue_dict[res]['energy_delta'] = residue_dict[res]['energy']['complex'] \
#                 - residue_dict[res]['energy']['unbound']  # - residue_dict[res]['energy']['fsp']
#             rel_oligomer_sasa = calc_relative_sa(residue_dict[res]['type'],
#                                                  residue_dict[res]['sasa']['total']['unbound'])
#             rel_complex_sasa = calc_relative_sa(residue_dict[res]['type'],
#                                                 residue_dict[res]['sasa']['total']['complex'])
#             for polarity in residue_dict[res]['sasa']:
#                 # convert sasa measurements into bsa measurements
#                 residue_dict[res]['bsa_%s' % polarity] = round(residue_dict[res]['sasa'][polarity]['unbound']
#                                                                - residue_dict[res]['sasa'][polarity]['complex'], 2)
#             if residue_dict[res]['bsa_total'] > 0:
#                 if rel_oligomer_sasa < 0.25:
#                     residue_dict[res]['support'] = 1
#                 elif rel_complex_sasa < 0.25:
#                     residue_dict[res]['core'] = 1
#                 else:
#                     residue_dict[res]['rim'] = 1
#             else:  # Todo remove res from dictionary as no interface design should be done? keep interior res constant?
#                 if rel_complex_sasa < 0.25:
#                     residue_dict[res]['interior'] = 1
#                 # else:
#                 #     residue_dict[res]['surface'] = 1

#             residue_dict[res].pop('sasa')
#             residue_dict[res].pop('energy')
#             # if residue_dict[res]['energy'] <= hot_spot_energy:
#             #     residue_dict[res]['hot_spot'] = 1
        total_residue_dict[entry] = residue_dict

    return total_residue_dict

In [123]:
old_score_dict = read_scores('/home/kylemeador/designs/2gtr_2dj6/DEGEN_1_2/ROT_19_29/tx_72/scores/design_scores_without_sym_residues.sc')
sym_new_score_dict = read_scores('/home/kylemeador/designs/2gtr_2dj6/DEGEN_1_2/ROT_19_29/tx_72/scores/design_scores_with_only_design_sym_residues.sc')
correct_asu_sym_new_score_dict = read_scores('/home/kylemeador/designs/2gtr_2dj6/DEGEN_1_2/ROT_19_29/tx_72/scores/design_scores_correct_asu.sc')
correct_asu_old_score_dict = read_scores('/home/kylemeador/designs/2gtr_2dj6/DEGEN_1_2/ROT_19_29/tx_72/scores/design_scores_correct_asu_no_sym.sc')

In [124]:
old_residues = dirty_dirty_residue_processing(old_score_dict)
sym_new_residues = dirty_dirty_residue_processing(sym_new_score_dict)
correct_asu_sym_new_residues = dirty_dirty_residue_processing(correct_asu_sym_new_score_dict) 
correct_asu_old_residues = dirty_dirty_residue_processing(correct_asu_old_score_dict)

In [113]:
# old_residues
# sym_new_residues
new_d, same_d, different_d = {}, {}, {}
for design in old_residues:
    new_d[design] = {residue: {'energy_sym': residue_info['energy']['complex'], 
                               'sasa_sym': {'polar': residue_info['sasa']['polar']['complex'],
                                        'hydrophobic': residue_info['sasa']['hydrophobic']['complex'],
                                        'total': residue_info['sasa']['total']['complex']}
                              } 
                     for residue, residue_info in sym_new_residues[design].items()}
    same_d[design], different_d[design] = {}, {}
    for residue, residue_info in old_residues[design].items():
        new_d[design][residue]['energy'] = residue_info['energy']['complex']
        new_d[design][residue]['sasa'] = {'polar': residue_info['sasa']['polar']['complex'],
                                          'hydrophobic': residue_info['sasa']['hydrophobic']['complex'],
                                          'total': residue_info['sasa']['total']['complex']}
    for residue, residue_info in new_d[design].items():
        if 'energy_sym' in residue_info and 'energy' in residue_info:  # new_d[design][residue] and 'energy' in new_d[design][residue]:
            same_d[design][residue] = new_d[design][residue]
        else:
            different_d[design][residue] = new_d[design][residue]

In [137]:
# print(correct_asu_sym_new_residues)
clean_correct_asu_sym_d, clean_same_d, clean_different_d = {}, {}, {}
for design in correct_asu_sym_new_residues:
    clean_correct_asu_sym_d[design] = {residue: {'energy_sym': residue_info['energy']['complex'], 
                                                 'sasa_sym': {'polar': residue_info['sasa']['polar']['complex'],
                                                              'hydrophobic': residue_info['sasa']['hydrophobic']['complex'],
                                                              'total': residue_info['sasa']['total']['complex']}
                                                }
                                       for residue, residue_info in correct_asu_sym_new_residues[design].items()}
    for residue, residue_info in correct_asu_old_residues[design].items():
        clean_correct_asu_sym_d[design][residue]['energy'] = residue_info['energy']['complex']
        clean_correct_asu_sym_d[design][residue]['sasa'] = {'polar': residue_info['sasa']['polar']['complex'],
                                                            'hydrophobic': residue_info['sasa']['hydrophobic']['complex'],
                                                            'total': residue_info['sasa']['total']['complex']}
    clean_same_d[design], clean_different_d[design] = {}, {}
    for residue, residue_info in clean_correct_asu_sym_d[design].items():
        if 'energy_sym' in residue_info and 'energy' in residue_info:  # new_d[design][residue] and 'energy' in new_d[design][residue]:
            clean_same_d[design][residue] = clean_correct_asu_sym_d[design][residue]
        else:
            clean_different_d[design][residue] = clean_correct_asu_sym_d[design][residue]

In [114]:
# OLD VERSION
clean_correct_asu_sym_d = {design: 
                           {residue: {'energy_sym': residue_info['energy']['complex'], 
                                      'sasa_sym': {'polar': residue_info['sasa']['polar']['complex'],
                                                   'hydrophobic': residue_info['sasa']['hydrophobic']['complex'],
                                                   'total': residue_info['sasa']['total']['complex']}
                                     }
                            for residue, residue_info in residue_data.items()}
                           for design, residue_data in correct_asu_sym_new_residues.items()}
clean_same_d = {design: {residue: data for residue, data in residue_info.items() if residue in same_d['clean_asu_for_refine_design_0001']} 
                for design, residue_info in clean_correct_asu_sym_d.items()}  
clean_different_d = {design: {residue: data for residue, data in residue_info.items() if residue not in same_d['clean_asu_for_refine_design_0001']} 
                for design, residue_info in clean_correct_asu_sym_d.items()}

In [87]:
new_residue_d = {}
second_chain = 249
for residue, info in sym_new_residues[design].items():
    if residue % 353 >= second_chain:
        chain_idx = (int(residue / 353) * 2) + 1
    else:
        chain_idx = int(residue / 353) * 2
    chain = PDB.available_letters[chain_idx]
    new_residue_d[residue] = '%s%d' % (chain, residue % 353)
#     else:
#         new_residue_d[residue] = '%s%d' % (chain, residue % 353)

In [115]:
# new_residue_d
modify_dictionaries = [new_d, same_d, different_d]
for dict_ in modify_dictionaries:
    for design, residue_info in list(dict_.items()):
        for residue_number, data in list(residue_info.items()):
            dict_[design][new_residue_d[residue_number]] = dict_[design].pop(residue_number)

In [138]:
modify_dictionaries = [clean_correct_asu_sym_d, clean_same_d, clean_different_d]
for dict_ in modify_dictionaries:
    for design, residue_info in list(dict_.items()):
        for residue_number, data in list(residue_info.items()):
            dict_[design][new_residue_d[residue_number]] = dict_[design].pop(residue_number)

In [117]:
print(len(clean_same_d['clean_asu_for_refine_design_correct_asu_0001']))
print(len(same_d['clean_asu_for_refine_design_0001']))

76
76


In [171]:
sum_energy_sym, sum_energy = {}, {}
for design, residue_info in same_d.items():
    sum_energy_sym[design], sum_energy[design] = 0, 0
    for residue, data in residue_info.items():
        sum_energy_sym[design] += data['energy_sym']
        sum_energy[design] += data['energy']
        
sum_energy_sym_diff = {}
for design, residue_info in different_d.items():
    sum_energy_sym_diff[design] = 0
    for residue, data in residue_info.items():
        sum_energy_sym_diff[design] += data['energy_sym']

In [132]:
print(clean_correct_asu_sym_d)

{}


In [172]:
clean_sum_energy_sym_same, clean_sum_energy_all = {}, {}
for design, residue_info in clean_same_d.items():
    clean_sum_energy_sym_same[design], clean_sum_energy_all[design] = 0, 0
    for residue, data in residue_info.items():
        clean_sum_energy_sym_same[design] += data['energy_sym']
        clean_sum_energy_all[design] += data['energy']
        

clean_sum_energy_sym_different = {}
for design, residue_info in clean_different_d.items():
    clean_sum_energy_sym_different[design] = 0
    for residue, data in residue_info.items():
        clean_sum_energy_sym_different[design] += data['energy_sym']
        
clean_sum_energy_sym_all = {}
for design, residue_info in clean_correct_asu_sym_d.items():
    clean_sum_energy_sym_all[design] = 0
    for residue, data in residue_info.items():
        clean_sum_energy_sym_all[design] += data['energy_sym']
        
print(design)
print(clean_sum_energy_sym_all[design])
print(clean_sum_energy_sym_different[design])
print(clean_sum_energy_sym_same[design])
print('Not Symmetric')
print(clean_sum_energy_all[design])
print(sum_energy_sym['clean_asu_for_refine_design_0001'])
print(sum_energy_sym_diff['clean_asu_for_refine_design_0001'])
print(sum_energy['clean_asu_for_refine_design_0001'])

clean_asu_for_refine_design_correct_asu_0001
-270.34800000000007
-2.35
-267.99800000000005
Not Symmetric
-267.99800000000005
-218.44300000000007
-51.90199999999998
-205.208


In [119]:
print([(residue, data) for residue, data in clean_different_d[design].items() if abs(data['energy_sym']) > 0])

[('L312', {'energy_sym': -0.116, 'sasa_sym': {'polar': 17.686, 'hydrophobic': 18.961, 'total': 36.647}}), ('L314', {'energy_sym': -1.081, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 18.444, 'total': 18.444}}), ('M44', {'energy_sym': -0.305, 'sasa_sym': {'polar': 35.273, 'hydrophobic': 25.071, 'total': 60.344}}), ('N281', {'energy_sym': -0.04, 'sasa_sym': {'polar': 35.285, 'hydrophobic': 16.046, 'total': 51.331}}), ('N282', {'energy_sym': -0.512, 'sasa_sym': {'polar': 6.046, 'hydrophobic': 1.537, 'total': 7.583}}), ('N283', {'energy_sym': -0.366, 'sasa_sym': {'polar': 0.794, 'hydrophobic': 3.076, 'total': 3.871}}), ('N285', {'energy_sym': 0.076, 'sasa_sym': {'polar': 0.608, 'hydrophobic': 0.0, 'total': 0.608}}), ('N288', {'energy_sym': 0.005, 'sasa_sym': {'polar': 23.035, 'hydrophobic': 73.733, 'total': 96.768}}), ('N291', {'energy_sym': -0.013, 'sasa_sym': {'polar': 28.521, 'hydrophobic': 16.079, 'total': 44.6}}), ('E234', {'energy_sym': 0.003, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 0

In [86]:
print(sum_energy_sym)
print(sum_energy)

{'clean_asu_for_refine': -101.84899999999998, 'clean_asu_for_refine_design_0001': -218.44300000000007, 'clean_asu_for_refine_design_0002': -213.414, 'clean_asu_for_refine_design_0003': -237.09699999999992, 'clean_asu_for_refine_design_0004': -224.85000000000005, 'clean_asu_for_refine_design_0005': -199.25999999999996, 'clean_asu_for_refine_design_0006': -210.45799999999997}
{'clean_asu_for_refine': -78.47000000000001, 'clean_asu_for_refine_design_0001': -205.208, 'clean_asu_for_refine_design_0002': -201.921, 'clean_asu_for_refine_design_0003': -227.22100000000003, 'clean_asu_for_refine_design_0004': -212.59599999999992, 'clean_asu_for_refine_design_0005': -184.35900000000004, 'clean_asu_for_refine_design_0006': -199.05400000000006}


In [54]:
# len(same_d['clean_asu_for_refine'])
# same_d['clean_asu_for_refine']
impacted_sym_residues = {}
for residue, data in different_d['clean_asu_for_refine_design_0001'].items():
    if abs(data['energy_sym']) > 0:
        impacted_sym_residues[residue] = data

In [55]:
print(list(impacted_sym_residues.keys()))
print([impacted_sym_residues[residue]['energy_sym'] for residue in impacted_sym_residues])

['J281', 'J282', 'J283', 'J285', 'J288', 'J291', 'R280', 'R281', 'R282', 'R283', 'R285', 'R286', 'R288', 'R289', 'R290', 'R291', 'R292', 'R293', 'R294', 'R295', 'R296', 'R297', 'R299', 'R300', 'R302', 'R309', 'R321', 'R322', 'R324', 'R325', 'R326', 'R327', 'R328', 'R329', 'R330', 'R331', 'R332', 'R333', 'T312', 'T314', 'W29', 'W30', 'W32', 'W33', 'W34', 'W36', 'W37', 'W40', 'W41', 'W43', 'W44', 'W45', 'W71', 'W75', 'W77', 'W78', 'W79', 'W81', 'W82', 'W83', 'W84', 'W85', 'W86', 'W88', 'W89', 'W90', 'W92', 'W93', 'W96', 'W224', 'W227', 'W228', 'W231', 'W232', 'W235', 'W236', 'W239', 'D312', 'D314', 'E44', 'E234', 'E236']
[-0.04, -0.512, -0.366, 0.076, 0.005, -0.013, -0.3, -0.825, -0.033, -0.607, -1.877, -1.209, -1.728, -0.715, -0.281, -0.517, -1.658, -0.718, -0.02, -0.519, -1.026, -0.604, -1.844, -0.684, -0.026, -0.958, -0.435, -0.017, -0.558, -1.029, -0.023, -0.049, -2.923, -1.43, -0.47, -0.579, -0.446, -0.065, -0.068, -0.823, -0.517, -0.002, -0.006, -0.269, 0.002, -0.339, -1.722, -1.22

In [149]:
# same_d['clean_asu_for_refine_design_0001']
target_sym_residues = ['R280', 'R281', 'R282', 'R283', 'R285', 'R286', 'R288', 'R289', 'R290', 'R291', 'R292', 'R293', 'R294', 'R295', 'R296', 'R297', 'R299', 'R300', 'R302', 'R309', 'D312', 'D314']
asu_check = [same_d['clean_asu_for_refine_design_0001'][residue.replace('R', 'B').replace('D', 'B')]['energy_sym'] 
             for residue in target_sym_residues]
non_sym_asu = [same_d['clean_asu_for_refine_design_0001'][residue.replace('R', 'B').replace('D', 'B')]['energy'] 
               for residue in target_sym_residues]
clean_sym_asu = [clean_same_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('R', 'B').replace('D', 'B')]['energy_sym'] 
               for residue in target_sym_residues]
clean_asu = [clean_same_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('R', 'B').replace('D', 'B')]['energy'] 
               for residue in target_sym_residues]

In [156]:
print([(letter, clean_different_d['clean_asu_for_refine_design_correct_asu_0001']['%s314' % letter]) for letter in PDB.available_letters[3:24:2]])

[('D', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 21.776, 'total': 21.776}}), ('F', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 20.239, 'total': 20.239}}), ('H', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 19.214, 'total': 19.214}}), ('J', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 12.296, 'total': 12.296}}), ('L', {'energy_sym': -1.081, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 18.444, 'total': 18.444}}), ('N', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 16.14, 'total': 16.14}}), ('P', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 13.066, 'total': 13.066}}), ('R', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 10.247, 'total': 10.247}}), ('T', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.675, 'hydrophobic': 15.37, 'total': 16.045}}), ('V', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 13.833, 'total': 13.833}}), ('X', {'energy_sym': 0.0, 'sasa_sym':

In [159]:
clean_asu_different = [clean_different_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('D', 'L')]['energy_sym']
                       if residue in clean_different_d['clean_asu_for_refine_design_correct_asu_0001'] else 0
                       for residue in target_sym_residues]

In [179]:
# sym_energy_residue_check = [impacted_sym_residues[residue]['energy_sym'] for residue in impacted_sym_residues
#                             if residue in target_sym_residues]
sym_energy_residue_check = [data['energy_sym'] for residue, data in different_d['clean_asu_for_refine_design_0001'].items()
                            if residue in target_sym_residues]
print('    NO clean dirty  SYM clean_asu dirty_asu MATES clean_asu_sym dirty_sym_sym')
for idx, value in enumerate(sym_energy_residue_check):
    print('%s| %.3f %.3f %s    %.3f    %.3f %s     %.3f        %.3f' % (target_sym_residues[idx].replace('R', 'B').replace('D', 'B'), clean_asu[idx], non_sym_asu[idx], ' | ', clean_sym_asu[idx], asu_check[idx], '  |      ', clean_asu_different[idx], value))

    NO clean dirty  SYM clean_asu dirty_asu MATES clean_asu_sym dirty_sym_sym
B280| -1.297 -0.982  |     -1.297    -0.997   |           0.000        -0.300
B281| -2.534 -1.662  |     -2.534    -1.710   |           0.000        -0.825
B282| -2.366 -2.260  |     -2.366    -2.333   |           0.000        -0.033
B283| -1.959 -1.028  |     -1.959    -1.351   |           0.000        -0.607
B285| -1.196 1.322  |     -1.196    0.682   |           0.000        -1.877
B286| -4.071 -2.565  |     -4.071    -2.862   |           0.000        -1.209
B288| -1.707 0.092  |     -1.707    0.021   |           0.000        -1.728
B289| -2.960 -1.904  |     -2.960    -2.246   |           0.000        -0.715
B290| -4.345 -3.910  |     -4.345    -4.064   |           0.000        -0.281
B291| -5.778 -5.151  |     -5.778    -5.261   |           0.000        -0.517
B292| -5.224 -3.354  |     -5.224    -3.566   |           0.000        -1.658
B293| -5.023 -4.196  |     -5.023    -4.305   |           0.000     

For the original input, with the asu chains separated by a random amount:
B matches with J, T, and W
A matches with C (same), E (same), and R

In [67]:
print(len(sym_new_residues[design]) / 12)  # all instances of residue measurement
print(len(same_d[design]))  # all the same (ASU)

76.0
76


Now I am checking for the SASA measurement

In [188]:
# same_d['clean_asu_for_refine_design_0001']
target_sym_residues = ['R280', 'R281', 'R282', 'R283', 'R285', 'R286', 'R288', 'R289', 'R290', 'R291', 'R292', 'R293', 'R294', 'R295', 'R296', 'R297', 'R299', 'R300', 'R302', 'R309', 'D312', 'D314']
clean_asu_sasa = [clean_same_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('R', 'B').replace('D', 'B')]['sasa'] 
                  for residue in target_sym_residues]
asu_sasa = [same_d['clean_asu_for_refine_design_0001'][residue.replace('R', 'B').replace('D', 'B')]['sasa'] 
                    for residue in target_sym_residues]
clean_sym_asu_sasa = [clean_same_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('R', 'B').replace('D', 'B')]['sasa_sym'] 
                      for residue in target_sym_residues]
asu_sym_sasa = [same_d['clean_asu_for_refine_design_0001'][residue.replace('R', 'B').replace('D', 'B')]['sasa_sym'] 
                  for residue in target_sym_residues]

In [156]:
print([(letter, clean_different_d['clean_asu_for_refine_design_correct_asu_0001']['%s314' % letter]) for letter in PDB.available_letters[3:24:2]])

[('D', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 21.776, 'total': 21.776}}), ('F', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 20.239, 'total': 20.239}}), ('H', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 19.214, 'total': 19.214}}), ('J', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 12.296, 'total': 12.296}}), ('L', {'energy_sym': -1.081, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 18.444, 'total': 18.444}}), ('N', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 16.14, 'total': 16.14}}), ('P', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 13.066, 'total': 13.066}}), ('R', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 10.247, 'total': 10.247}}), ('T', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.675, 'hydrophobic': 15.37, 'total': 16.045}}), ('V', {'energy_sym': 0.0, 'sasa_sym': {'polar': 0.0, 'hydrophobic': 13.833, 'total': 13.833}}), ('X', {'energy_sym': 0.0, 'sasa_sym':

In [189]:
clean_asu_different_sasa = [clean_different_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('D', 'L')]['sasa_sym']
                            if residue in clean_different_d['clean_asu_for_refine_design_correct_asu_0001'] else 0
                            for residue in target_sym_residues]
clean_asu_different_sasa2 = [clean_different_d['clean_asu_for_refine_design_correct_asu_0001'][residue.replace('R', 'D')]['sasa_sym']
                             if residue in clean_different_d['clean_asu_for_refine_design_correct_asu_0001'] else 0
                             for residue in target_sym_residues]

In [191]:
# sym_energy_residue_check = [impacted_sym_residues[residue]['energy_sym'] for residue in impacted_sym_residues
#                             if residue in target_sym_residues]
sym_energy_residue_check = [data['sasa_sym'] for residue, data in different_d['clean_asu_for_refine_design_0001'].items()
                            if residue in target_sym_residues]
for sasa_type in ['total']:  # , 'polar', 'hydrophobic']:
    print(sasa_type)
    print('    NO clean dirty  SYM clean_asu dirty_asu MATES clean_asu_sym dirty_sym_sym |  clean_different')
    for idx, value in enumerate(sym_energy_residue_check):
        print('%s| %.3f %.3f %s    %.3f    %.3f %s     %.3f        %.3f |   %.3f' 
              % (target_sym_residues[idx].replace('R', 'B').replace('D', 'B'), 
                 clean_asu_sasa[idx][sasa_type], asu_sasa[idx][sasa_type], 
                 ' | ', clean_sym_asu_sasa[idx][sasa_type], asu_sym_sasa[idx][sasa_type], 
                 '  |      ', clean_asu_different_sasa[idx][sasa_type], value[sasa_type], clean_asu_different_sasa2[idx][sasa_type]))

total
    NO clean dirty  SYM clean_asu dirty_asu MATES clean_asu_sym dirty_sym_sym |  clean_different
B280| 71.485 67.891  |     71.485    67.891   |           73.787        71.485 |   75.069
B281| 51.951 49.681  |     51.951    49.681   |           59.264        51.951 |   55.093
B282| 6.396 8.543  |     6.396    8.543   |           12.809        6.396 |   13.744
B283| 4.871 7.459  |     4.871    7.459   |           1.025        4.871 |   3.613
B285| 1.216 1.824  |     1.216    1.824   |           0.000        1.216 |   1.216
B286| 22.295 19.474  |     22.295    20.498   |           15.632        22.295 |   22.040
B288| 115.117 101.556  |     115.117    101.556   |           93.232        115.117 |   110.025
B289| 0.000 0.000  |     0.000    0.000   |           0.000        0.000 |   0.000
B290| 0.000 0.000  |     0.000    0.000   |           0.000        0.000 |   1.025
B291| 53.219 55.422  |     53.219    54.652   |           43.679        53.219 |   56.222
B292| 0.000 0.000  |    

In [193]:
symmetric_test_analysis_file = '/home/kylemeador/designs/metrics_testing_analysis.csv'
sym_test_df = pd.read_csv(symmetric_test_analysis_file, index_col=0, header=[0,1,2])

In [195]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
mean = 'mean'
std = 'std'
idx = pd.IndexSlice

In [196]:
sym_test_df

design_profile             \
                                                   energy_distance       mean   
                                                     no_constraint buns_total   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...        8.249736   4.333333   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...        8.349393  11.333333   
Projects-metrics_testing_Designs-outF_10                  7.485495   2.000000   
Projects-metrics_testing_Designs-outF_10-odd-asu          8.001939   2.000000   

                                                                  \
                                                                   
                                                   contact_count   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...      4.890231   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...      4.908106   
Projects-metrics_testing_Designs-outF_10                4.886904   
Projects-metrics_testing_Designs-outF_10-odd-asu        4.886904   

                                                                         \
                                                                          
                                                   interface_area_total   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...          2440.944550   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...          2256.948615   
Projects-metrics_testing_Designs-outF_10                     907.249570   
Projects-metrics_testing_Designs-outF_10-odd-asu             895.803111   

                                                                     \
                                                                      
                                                   interface_energy   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...       -61.491881   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...       -61.227145   
Projects-metrics_testing_Designs-outF_10                 -27.682385   
Projects-metrics_testing_Designs-outF_10-odd-asu         -27.682385   

                                                                  \
                                                                   
                                                   number_hbonds   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...           6.0   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...           6.0   
Projects-metrics_testing_Designs-outF_10                     0.0   
Projects-metrics_testing_Designs-outF_10-odd-asu             0.0   

                                                                                       \
                                                                                        
                                                   percent_interface_area_hydrophobic   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                           0.796724   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                           0.796809   
Projects-metrics_testing_Designs-outF_10                                     0.865286   
Projects-metrics_testing_Designs-outF_10-odd-asu                             0.866821   

                                                                          \
                                                                           
                                                   shape_complementarity   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...              0.673831   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...              0.673751   
Projects-metrics_testing_Designs-outF_10                        0.711750   
Projects-metrics_testing_Designs-outF_10-odd-asu                0.710660   

                                                                     \
                                                                      
                                                   solvation_energy   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...     1.421085e-14   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...    -2.842171e-

TO SOLVE
Buns_total is very incorrect somehow. Higher values with good ASU input. I think this is the last straggler that rosetta_scripts needs tweaking
STDEV is larger with the correct-asu interface_area_total, interface_energy. Is the PoseDeletionMover working correctly?

Checking out Trajectories.csv / Residues.csv to understand
Energy of residues may be slightly off with symmetry
Interface area is messed up slightly. Think error is more than orientational - Chain B values are the only ones affected!

SOLVED
interface_energy may be off (different packing in metrics could be the cause. yes POSE metrics would argue so. Bound is the exact same, while unbound (packed) have changed)
Solvation energy is missing. interaction_energy_complex = -81.018555, interface_energy_no_intra_residue_score = -81.018555, interface_energy = -66.49 (this is old delta) - NOT actually calculating
shape_complementarity is slightly different with the correct versus odd-asu. Probably due to molecular dots issues


RENAME int_composition_similarity to full "interface". ref to rosetta_reference_energy
Names of all PERCENT_ columns with 0,1 bound as percent?

!!Compute the delta of each interface upon binding. 
For total, this the interaction_energy_complex - interface_energy
For each individual bound - unbound = positive value (transition energy to reach the complex)!!

residue_type classification is different depending on the values of SASA (I believe) which causes issues in all of these!
!!Pose mean total_interface_residues is different (for 2gtr 57 with odd-asu, 55 with correct). Dock total_interface_residues is 76!!
The interior residues seem to be the difference between these two...
nanohedra metric "percent_fragment" is different in 2gtr. Higher in correct despite no difference in other metrics? 
divergence (design/evolution) is slightly higher with correct ASU configuration (fewer residues)
divergence of fragment information is missing (observed_fragment too)
observed_design/evolution are the same. Do they use non-interior residues?

maybe remove:
interface_energy_no_intra_residue_score, interface_energy_unbound
remove:
cst_weight, fsp_energy, int_area_res_summary_hydrophobic_1/2_unbound, int_area_res_summary_polar_1/2_unbound, int_area_res_summary_total_1/2_unbound, int_area_total/polar/hydrophobic, 
int_energy_context_1/2_unbound, int_energy_res_summary_1/2_unbound, int_energy_res_summary_complex, int_sc, int_sc_median_dist, 

In [199]:
symmetric_test_analysis_file = '/home/kylemeador/designs/metrics_testing_analysisV2.csv'
sym_test_df2 = pd.read_csv(symmetric_test_analysis_file, index_col=0, header=[0,1,2])

In [200]:
sym_test_df2

design_profile  \
                                                   energy_distance   
                                                     no_constraint   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...        8.578222   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...        8.540535   
Projects-metrics_testing_Designs-outF_10                 10.413234   
Projects-metrics_testing_Designs-outF_10-odd-asu         10.633061   

                                                                  \
                                                            mean   
                                                   contact_count   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...      4.890231   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...      4.908106   
Projects-metrics_testing_Designs-outF_10                3.024735   
Projects-metrics_testing_Designs-outF_10-odd-asu        3.010601   

                                                                         \
                                                                          
                                                   interface_area_total   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...          2440.944550   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...          2256.948615   
Projects-metrics_testing_Designs-outF_10                    1015.105597   
Projects-metrics_testing_Designs-outF_10-odd-asu            1056.620184   

                                                                            \
                                                                             
                                                   interface_buried_hbonds   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                4.333333   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                6.000000   
Projects-metrics_testing_Designs-outF_10                          0.000000   
Projects-metrics_testing_Designs-outF_10-odd-asu                  0.000000   

                                                                     \
                                                                      
                                                   interface_energy   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...       -31.767482   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...       -31.120010   
Projects-metrics_testing_Designs-outF_10                   8.398264   
Projects-metrics_testing_Designs-outF_10-odd-asu           9.313586   

                                                                  \
                                                                   
                                                   number_hbonds   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...           6.0   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...           6.0   
Projects-metrics_testing_Designs-outF_10                     2.0   
Projects-metrics_testing_Designs-outF_10-odd-asu             2.0   

                                                                                       \
                                                                                        
                                                   percent_interface_area_hydrophobic   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                           0.796724   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...                           0.796809   
Projects-metrics_testing_Designs-outF_10                                     0.839608   
Projects-metrics_testing_Designs-outF_10-odd-asu                             0.844205   

                                                                          \
                                                                           
                                                   shape_complementarity   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...              0.673831   
Projects-metrics_testing_Designs-2gtr_2dj6-DEGE...              0.673751   
Projects-metrics_testing_Designs

Why increase in interface_area_total for outF_10? New interface was defined and it was not limited to the residues identified by Roger...

Interface energy went down significantly despite remaining the same between the two input styles. Did repacking with backbone flexibility really decrease the energy that drastically? Why is this the case?